In [1]:
import cv2
import pandas as pd
from ultralytics import YOLO
from moviepy import VideoFileClip, concatenate_videoclips
import numpy as np
import os

# --- KONFIGŪRACIJA ---
VIDEO_PATH = "data/Lakers_vs_Thunder.mp4"
OUTPUT_FOLDER = "results"
OUTPUT_PATH = os.path.join(OUTPUT_FOLDER, "summary.mp4")
CSV_PATH = os.path.join(OUTPUT_FOLDER, "analysis_data.csv")
MODEL_NAME = "models/yolo11s.pt"
TARGET_DURATION = 120
SCAN_INTERVAL = 10

In [2]:
def analyze_video(video_path, model):
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"KLAIDA: Video failas nerastas! Tikrinta vieta: {os.path.abspath(video_path)}")

    print(f"Pradedama video analizė: {video_path}")
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise ValueError("KLAIDA: OpenCV negali atidaryti video failo. Failas gali būti sugadintas arba netinkamo formato.")

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    yolo = YOLO(model)
    data = []

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % SCAN_INTERVAL == 0:
            results = yolo(frame, verbose=False, classes=[0, 32])

            person_count = 0
            ball_detected = 0

            for r in results:
                boxes = r.boxes
                for box in boxes:
                    cls = int(box.cls[0])
                    if cls == 0:
                        person_count += 1
                    elif cls == 32:
                        ball_detected = 1

            score = (person_count * 1) + (ball_detected * 50)

            data.append({
                'frame': frame_idx,
                'time_sec': frame_idx / fps,
                'score': score
            })

            if len(data) % 10 == 0:
                print(f"Progresas: {frame_idx}/{total_frames} | Balas: {score}...", end='\r')

        frame_idx += 1

    cap.release()
    df = pd.DataFrame(data)
    print("\nAnalizė baigta.")
    return df, fps


In [3]:
def add_audio_score(video_path, df):
    """
    Nuskaito video garso takelį ir prideda 'audio_score' stulpelį į DataFrame.
    """
    print("Vykdoma garso analizė (Audio Intensity)...")

    try:
        clip = VideoFileClip(video_path)
        audio = clip.audio

        audio_fps = 22050
        sound_array = audio.to_soundarray(fps=audio_fps)

        if sound_array.ndim == 2:
            sound_array = sound_array.mean(axis=1)

        total_samples = len(sound_array)
        duration = clip.duration

        audio_scores = []

        for t in df['time_sec']:
            start_t = max(0, t - 0.25)
            end_t = min(duration, t + 0.25)

            start_idx = int(start_t * audio_fps)
            end_idx = int(end_t * audio_fps)

            start_idx = max(0, start_idx)
            end_idx = min(total_samples, end_idx)

            chunk = sound_array[start_idx:end_idx]

            if len(chunk) > 0:
                rms = np.sqrt(np.mean(chunk**2))
            else:
                rms = 0

            audio_scores.append(rms)

        df['audio_raw'] = audio_scores

        # 1. Normalizuojame vaizdą
        v_min, v_max = df['score'].min(), df['score'].max()
        df['visual_norm'] = (df['score'] - v_min) / (v_max - v_min + 1e-6) # +1e-6 kad nedalintume iš 0

        # 2. Normalizuojame garsą
        a_min, a_max = df['audio_raw'].min(), df['audio_raw'].max()
        df['audio_norm'] = (df['audio_raw'] - a_min) / (a_max - a_min + 1e-6)

        df['final_score'] = (df['visual_norm'] * 0.6) + (df['audio_norm'] * 0.4)

        print("Garso analizė baigta.")
        return df

    except Exception as e:
        print(f"Klaida analizuojant garsą: {e}")
        # Jei nepavyksta, grąžiname originalą su 'final_score' = 'score'
        df['final_score'] = df['score']
        return df

In [4]:
def create_summary_advanced(video_path, df, target_duration):
    if df.empty:
        return

    print("Generuojamas montažas (pagal vaizdą ir garsą)...")

    # Imame viršutinį 10-15% geriausių momentų (priklausomai nuo video ilgio)
    threshold = df['final_score'].quantile(0.85)

    important_frames = df[df['final_score'] >= threshold].copy()

    # Buferiai (laikas prieš ir po svarbaus momento)
    BUFFER_BEFORE = 2.0
    BUFFER_AFTER = 2.0

    intervals = []
    for _, row in important_frames.iterrows():
        start = max(0, row['time_sec'] - BUFFER_BEFORE)
        end = row['time_sec'] + BUFFER_AFTER
        intervals.append([start, end])

    intervals.sort()

    merged_intervals = []
    if intervals:
        curr_start, curr_end = intervals[0]
        for next_start, next_end in intervals[1:]:
            # Jei tarpas tarp klipų mažesnis nei 1.5 sek, sujungiame juos
            if next_start <= curr_end + 1.5:
                curr_end = max(curr_end, next_end)
            else:
                merged_intervals.append((curr_start, curr_end))
                curr_start, curr_end = next_start, next_end
        merged_intervals.append((curr_start, curr_end))

    # Generavimas
    original_clip = VideoFileClip(video_path)
    subclips = []

    print(f"Atrinkta epizodų: {len(merged_intervals)}")

    current_duration = 0
    for start, end in merged_intervals:
        if current_duration >= target_duration:
            break

        end_time = min(original_clip.duration, end)
        if start < end_time:
            subclip = original_clip.subclipped(start, end_time)
            subclips.append(subclip)
            current_duration += (end_time - start)

    if subclips:
        final_clip = concatenate_videoclips(subclips)
        if final_clip.duration > target_duration:
            final_clip = final_clip.subclipped(0, target_duration)

        final_clip.write_videofile(OUTPUT_PATH, codec="libx264", audio_codec="aac")
        print(f"Baigta! Trukmė: {final_clip.duration:.2f}s")
    else:
        print("Nepavyko sukurti klipų.")

In [5]:
if __name__ == "__main__":
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    try:
        df_scores, fps_rate = analyze_video(VIDEO_PATH, MODEL_NAME)

        df_scores = add_audio_score(VIDEO_PATH, df_scores)

        df_scores.to_csv(CSV_PATH, index=False)
        print(f"Išsami analizė išsaugota: {CSV_PATH}")

        create_summary_advanced(VIDEO_PATH, df_scores, TARGET_DURATION)

    except Exception as e:
        print(f"Klaida: {e}")

Pradedama video analizė: data/Lakers_vs_Thunder.mp4
Progresas: 35490/35519 | Balas: 50...
Analizė baigta.
Vykdoma garso analizė (Audio Intensity)...
Garso analizė baigta.
Išsami analizė išsaugota: results/analysis_data.csv
Generuojamas montažas (pagal vaizdą ir garsą)...
Atrinkta epizodų: 12
MoviePy - Building video results/summary.mp4.
MoviePy - Writing audio in summaryTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video results/summary.mp4



MoviePy - Done !
MoviePy - video ready results/summary.mp4
Baigta! Trukmė: 120.00s
